In [122]:
import ee
import geemap
ee.Initialize()

In [123]:
Map = geemap.Map(center=[-11.77,-45.76], zoom=8)

# Define a region in which to generate a sample of the input.
region = geometry = ee.Geometry.Polygon(
        [[[-46.31633300781251, -12.379228237680417],
          [-45.31108398437501, -12.298734131305821],
          [-45.30009765625001, -11.589339623793398],
          [-46.33830566406251, -11.605482676116234]]])

Map

Map(center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [124]:
def clusterization(image):

          
    # Load a pre-computed Landsat composite for input.
    input = image.clip(geometry).select("EVI");
  
    #Make the training dataset.
    training = input.sample(**{
    'region': geometry,
    'scale': 250,
    'numPixels': 5000
    });

    #Instantiate the clusterer and train it.
    clusterer = ee.Clusterer.wekaKMeans(5).train(training)
    #Cluster the input using the trained clusterer.
    result = input.cluster(clusterer)
 
    #Combine the mean and standard deviation reducers.
    reducers = ee.Reducer.mean()

    #Inicialização Lista de EVI dos clustes
    lista = ee.List([])

    #Iteração sobre os 5 Clusters para calcular a estatística de cada região
    for i in range(5):

        #Cálculo da geometria
        geom = result.select("cluster").eq(i).selfMask().reduceToVectors(**{
        'scale':250,
        });



        #Reducer para cálculo do EVI médio de cada cluster
        stats = image.reduceRegion(**{
        'reducer': reducers,
        'geometry':geom.geometry(),
        'scale':250,
        'bestEffort':True,
        'maxPixels':1000000000,
        });

        #Armarzena o EVI médio de cada clúster em uma lista
        lista = lista.add(stats.get("EVI"));


    #Cópia da Lista para ordenar os 2 maiores EVIs
    lista2 = lista;


    #Ordenar a lista para pegar os 2 maiores valores de EVI médio nos clusters
    #Agora nossos clustests de interesse estão nas posições 3 e 4 da lista 2
    lista2 = lista2.sort()

    #Pegar as chaves dos clusteres de interesse na lista original
    chaves1 = lista.indexOf(lista2.get(3))
    chaves2 = lista.indexOf(lista2.get(4))


    #Cálculo da Geometria final

    geom1 = result.select("cluster").eq(chaves1).selfMask().reduceToVectors(**{
    'scale':250,
    });

    geom2 = result.select("cluster").eq(chaves2).selfMask().reduceToVectors(**{
    'scale':250,
    });

    #Geom é a feature collection de nossos clusters de interesse
    geom = geom1.merge(geom2);



    stats = image.reduceRegion(**{
    'reducer': reducers,
    'geometry':geom.geometry(),
    'scale':250,
    'bestEffort':True,
    'maxPixels':1000000000,
    });


    return image.set('EVI_Cluster', stats.get("EVI"))
    

In [125]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map

Map(center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [111]:
reducers = ee.Reducer.mean()


# Reducer para cálculo do EVI médio de cada cluster
stats = lst.reduceRegion(**{
  'reducer': reducers,
  'geometry':region,
  'scale':250,
});

NameError: name 'lst' is not defined

In [112]:
print(stats.getInfo())

NameError: name 'stats' is not defined

In [134]:
import requests
import pandas as pd
import json


In [150]:
ano = list(range(2020, 2022))
string_ints = [str(int) for int in ano]
stats = list();
    
for i in (string_ints):
    
    print (i)
#Filtrar a territorialidade 
    mcollection = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(geometry).filterDate( i + '-01-01', i +'-04-30').filter(ee.Filter.calendarRange(1, 4,'month')).map(clusterization)
        
    #4 principais meses de safra
    #filter start and end date    
    a = mcollection.aggregate_array('EVI_Cluster')

    x = a.getInfo()
    
    stats.append(x)    
    

2020
2021


In [151]:
stats

[[7385.27681330553,
  8495.536355505137,
  8148.69293869422,
  8456.67771961766,
  7373.441143668255,
  7075.421446839606,
  6314.883616130961,
  6648.418022290728],
 [7376.757254303962,
  8289.355512310685,
  8378.505444630135,
  7310.832362762665,
  7313.1025501202885,
  6883.330093942339,
  6782.831223628692,
  6608.550011879306]]

In [88]:
df = pd.DataFrame({'lin':x})

In [89]:
df

,lin
0,7385.276813
1,8495.536356
2,8148.692939
3,8456.677720
4,7373.441144
5,7075.421447
6,6314.883616
7,6648.418022
8,7376.757254
9,8289.355512


In [2]:
GEOCODE = '2919553'


In [3]:
url = f'https://apiprevmet3.inmet.gov.br/estacao/proxima/{GEOCODE}'
response = requests.get(url).json()
response

{'estacao': {'UF': 'BA',
  'CODIGO': 'A402',
  'LONGITUDE': '-45.02694443',
  'REGIAO': 'NE',
  'DISTANCIA_EM_KM': '105',
  'NOME': 'BARREIRAS',
  'LATITUDE': '-12.12472221',
  'GEOCODE': '2903201'},
 'dados': {'DC_NOME': 'BARREIRAS',
  'PRE_INS': None,
  'TEM_SEN': None,
  'VL_LATITUDE': '-12.12472221',
  'PRE_MAX': None,
  'UF': 'BA',
  'RAD_GLO': None,
  'PTO_INS': None,
  'TEM_MIN': None,
  'VL_LONGITUDE': '-45.02694443',
  'UMD_MIN': None,
  'PTO_MAX': None,
  'VEN_DIR': None,
  'DT_MEDICAO': '2021-07-21',
  'CHUVA': None,
  'PRE_MIN': None,
  'UMD_MAX': None,
  'VEN_VEL': None,
  'PTO_MIN': None,
  'TEM_MAX': None,
  'VEN_RAJ': None,
  'TEM_INS': None,
  'UMD_INS': None,
  'CD_ESTACAO': 'A402',
  'HR_MEDICAO': '1200'}}

In [8]:
estacao = response['dados']['CD_ESTACAO']

In [9]:
new_url = f'https://apitempo.inmet.gov.br/estacao/diaria/2000-01-01/2020-12-31/{estacao}'
json_response = requests.get(new_url).json()
dados_estacao = pd.DataFrame(json_response)
dados_estacao

,UMID_MED,DT_MEDICAO,DC_NOME,UMID_MIN,TEMP_MED,CHUVA,VL_LATITUDE,TEMP_MIN,TEMP_MAX,UF,VEL_VENTO_MED,CD_ESTACAO,VL_LONGITUDE
0,None,2000-01-01,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
1,None,2000-01-02,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
2,None,2000-01-03,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
3,None,2000-01-04,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
4,None,2000-01-05,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7666,74.2,2020-12-27,BARREIRAS,47,26.4,None,-12.12472221,21.2,32.2,BA,1.3,A402,-45.02694443
7667,73.3,2020-12-28,BARREIRAS,44,25.8,None,-12.12472221,19.5,32.3,BA,0.7,A402,-45.02694443
7668,72,2020-12-29,BARREIRAS,39,26.3,None,-12.12472221,20.3,33.6,BA,0.8,A402,-45.02694443
7669,66,2020-12-30,BARREIRAS,39,26.8,None,-12.12472221,20,33.9,BA,1,A402,-45.02694443


In [2]:
#API Estação INMET
#Recuperar dados da estação mais próxima ao geocode passado como parâmetro:
#https://apiprevmet3.inmet.gov.br/estacao/proxima/<geocode>

geocode = '2919553'

base_site = 'https://apiprevmet3.inmet.gov.br/estacao/proxima/' + geocode

r = requests.get(base_site)

json = r.json()

dt = pd.DataFrame(json['dados'],index=[0])

estacao = dt['CD_ESTACAO'].values[0]

base_site2 = 'https://apitempo.inmet.gov.br/estacao/diaria/2000-01-01/2020-12-31/' + estacao

r = requests.get(base_site2)

json = r.json()

dt2 = pd.DataFrame(json)


In [48]:
dados_estacao['CHUVA'] = dados_estacao['CHUVA'].astype(float)
dados_estacao['DT_MEDICAO'] = pd.to_datetime(dados_estacao['DT_MEDICAO'])
dados_estacao['ANO'] = dados_estacao['DT_MEDICAO'].dt.year
dados_estacao['MES'] = dados_estacao['DT_MEDICAO'].dt.month
dados_estacao

,UMID_MED,DT_MEDICAO,DC_NOME,UMID_MIN,TEMP_MED,CHUVA,VL_LATITUDE,TEMP_MIN,TEMP_MAX,UF,VEL_VENTO_MED,CD_ESTACAO,VL_LONGITUDE,ANO,MES
0,None,2000-01-01,BARREIRAS,None,None,NaN,-12.12472221,None,None,BA,None,A402,-45.02694443,2000,1
1,None,2000-01-02,BARREIRAS,None,None,NaN,-12.12472221,None,None,BA,None,A402,-45.02694443,2000,1
2,None,2000-01-03,BARREIRAS,None,None,NaN,-12.12472221,None,None,BA,None,A402,-45.02694443,2000,1
3,None,2000-01-04,BARREIRAS,None,None,NaN,-12.12472221,None,None,BA,None,A402,-45.02694443,2000,1
4,None,2000-01-05,BARREIRAS,None,None,NaN,-12.12472221,None,None,BA,None,A402,-45.02694443,2000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7666,74.2,2020-12-27,BARREIRAS,47,26.4,NaN,-12.12472221,21.2,32.2,BA,1.3,A402,-45.02694443,2020,12
7667,73.3,2020-12-28,BARREIRAS,44,25.8,NaN,-12.12472221,19.5,32.3,BA,0.7,A402,-45.02694443,2020,12
7668,72,2020-12-29,BARREIRAS,39,26.3,NaN,-12.12472221,20.3,33.6,BA,0.8,A402,-45.02694443,2020,12
7669,66,2020-12-30,BARREIRAS,39,26.8,NaN,-12.12472221,20,33.9,BA,1,A402,-45.02694443,2020,12


In [42]:
b  = dados_estacao.groupby(['ANO', 'MES'])['CHUVA'].mean().fillna(0)

ANO   MES
2000  1      0.00
      2      0.00
      3      0.00
      4      0.00
      5      0.00
             ... 
2020  8      0.00
      9      0.02
      10     0.00
      11     0.00
      12     0.00
Name: CHUVA, Length: 252, dtype: float64

In [66]:
mask = (dados_estacao['MES'] <= 4) | (dados_estacao['MES'] >= 11)  
b = dados_estacao.loc[mask].groupby(['ANO', 'MES'])['CHUVA'].mean().fillna(0)

In [91]:
a = b[82:88]

4.207142857142857